In [ ]:
import os
import sys

import ipywidgets as widgets
from IPython.display import display
from IPython.display import update_display
from IPython.display import display_pretty
import psycopg2

%matplotlib notebook
import matplotlib.pyplot as plt

from gpt_api import set_openai_key, GPT, Example

In [ ]:
# Load credentials from environment. 
POSTGRES_ADDRESS = os.environ.get('POSTGRES_ADDRESS')
POSTGRES_PORT = os.environ.get('POSTGRES_PORT')
POSTGRES_USERNAME = os.environ.get('POSTGRES_USERNAME')
POSTGRES_PASSWORD = os.environ.get('POSTGRES_PASSWORD')
POSTGRES_DBNAME = os.environ.get('POSTGRES_DBNAME')
OPENAI_KEY = os.environ.get('OPENAI_KEY')

In [ ]:
# Create connection and cursor.
conn = psycopg2.connect(host=POSTGRES_ADDRESS,
                  database=POSTGRES_DBNAME,
                  user=POSTGRES_USERNAME,
                  password=POSTGRES_PASSWORD,
                  port=POSTGRES_PORT)
cur = conn.cursor()

In [ ]:
# Set OpenAI key.
set_openai_key(OPENAI_KEY)

In [ ]:
# Construct GPT-3-instruct instance, add instruction and examples
gpt = GPT(engine="davinci-instruct-beta",
          temperature=0.3,
          max_tokens=200)
gpt.add_instruction("Given an input question, respond with syntactically correct SQL. "
                    "Only use the table called 'income_table', 'profile_table', and 'balance'. "
                    "The 'income_table' table has columns: "
                    "symbol, date, revenue, grossProfit, costAndExpenses, researchAndDevelopmentExpenses, ebitda. "
                    "The 'profile_table' table has columns: "
                    "symbol, mktCap, price, description, ceo, address, ipoDate. "
                    "The 'balance' table has columns: symbol, date, cashAndCashEquivalents, "
                    "totalCurrentAssets, goodwill, totalInvestments, totalDebt.")

gpt.add_example(Example("What is the market cap of Google?", 
                        "SELECT mktCap FROM profile_table WHERE symbol = 'GOOGL'"))

gpt.add_example(Example("Who is Facebook's CEO?", 
                        "SELECT ceo FROM profile_table WHERE symbol = 'FB'"))

gpt.add_example(Example("What are the 5 companies that have the most assets?", 
                        "SELECT symbol, totalCurrentAssets from balance WHERE EXTRACT(YEAR FROM date) = 2020 ORDER BY totalCurrentAssets desc LIMIT 5;"))

gpt.add_example(Example("How have Nvidia's investments changed over the past 10 years?", 
                        "SELECT date, totalInvestments FROM balance WHERE symbol = 'NVDA'"
                        " AND date >= now() - interval '10 years'"))

gpt.add_example(Example("How much money did Amazon make in 2017?", 
                        "SELECT SUM(grossProfit) FROM income_table WHERE symbol = 'AMZN' AND"
                        " EXTRACT(YEAR FROM date) = 2017")) 

gpt.add_example(Example("What were the top 7 companies with the highest revenue?", 
                        "SELECT symbol, revenue from income_table WHERE EXTRACT(YEAR FROM date) = 2020 ORDER BY revenue desc LIMIT 7"))

gpt.add_example(Example("What were Apple's expenses over the last 8 years?", 
                        "SELECT date, costAndExpenses FROM income_table WHERE symbol = 'AAPL'"
                        " AND date >= now() - interval '8 years'"))

In [ ]:
# Define display flow for GPT question input
def on_button_clicked(b):
    def sql_button_clicked(b):
        conn.commit()
        cur.execute(query)
        result = cur.fetchall()
        if len(result) == 1:
            display(result[0][0])
        elif len(result) >= 1 and len(result[0]) == 2:
            # Display graph data
            %matplotlib inline
            x, y = zip(*result)
            plt.plot(x, y)
        else:
            display(result)
        
    print ('\033[1mInput:\033[0m ' + inp.value)
    output = gpt.submit_request(inp.value)
    result = output['choices'][0].text
    query = result.split('output:')[1]
    print ('\033[1mGPT-3 Response:\033[0m ' + query)
    button2 = widgets.Button(description="Run SQL")
    button2.on_click(sql_button_clicked)
    display(button2)

# Run the GPT-to-Postgres widget

Translate English-language questions into SQL queries and execute queries on the database.

In [ ]:
# Display UI to give GPT-3 prompt and run and display resulting SQL
inp = widgets.Text(description='Ask GPT-3:')
button1 = widgets.Button(description="Get Response")
Box = widgets.HBox([inp, button1])
print ('\033[1mInstruction:\033[0m ' + "Given an input question, respond with syntactically correct PostgreSQL.")
button1.on_click(on_button_clicked)
display(Box)